# PEFT 进阶操作

## 1. 自定义模型适配

In [ ]:
import torch
from torch import nn
from peft import LoraConfig, get_peft_model, PeftModel

In [ ]:
net1 = nn.Sequential(
    nn.Linear(10, 10),
    nn.ReLU(),
    nn.Linear(10, 2)
)
net1

In [ ]:
for name, param in net1.named_parameters():
    print(name)

In [ ]:
config = LoraConfig(target_modules=["0"])

In [ ]:
model1 = get_peft_model(net1, config)

In [ ]:
model1

## 2. 多适配器加载与切换

In [ ]:
net2 = nn.Sequential(
    nn.Linear(10, 10),
    nn.ReLU(),
    nn.Linear(10, 2)
)
net2

In [ ]:
config1 = LoraConfig(target_modules=["0"])
model2 = get_peft_model(net2, config1)
model2.save_pretrained("./loraA")

In [ ]:
config2 = LoraConfig(target_modules=["2"])
model2 = get_peft_model(net2, config2)
model2.save_pretrained("./loraB")

In [ ]:
net2 = nn.Sequential(
    nn.Linear(10, 10),
    nn.ReLU(),
    nn.Linear(10, 2)
)
net2

In [ ]:
model2 = PeftModel.from_pretrained(net2, model_id="./loraA/", adapter_name="loraA")
model2

In [ ]:
model2.load_adapter("./loraB/", adapter_name="loraB")
model2

In [ ]:
model2.active_adapter

In [ ]:
model2(torch.arange(0, 10).view(1, 10).float())

In [ ]:
for name, param in model2.named_parameters():
    print(name, param)

In [ ]:
for name, param in model2.named_parameters():
    if name in ["base_model.model.0.lora_A.loraA.weight", "base_model.model.0.lora_B.loraA.weight"]:
        param.data = torch.ones_like(param)

In [ ]:
model2(torch.arange(0, 10).view(1, 10).float())

In [ ]:
model2.set_adapter("loraB")

In [ ]:
model2.active_adapter

In [ ]:
model2(torch.arange(0, 10).view(1, 10).float())

## 3. 禁用适配器

In [ ]:
model2.set_adapter("loraA")

In [ ]:
model2(torch.arange(0, 10).view(1, 10).float())

In [ ]:
with model2.disable_adapter():
    print(model2(torch.arange(0, 10).view(1, 10).float()))